In [2]:
import pandas as pd
import pyarrow.parquet as pq

In [3]:
df_items = pq.read_table('Datos/dfitemsOpen_compr.parquet').to_pandas()

Borrando las filas donde los valores de todas las columnas sean nulos.

In [4]:
df_items = df_items.dropna(how='all')

In [5]:
df_items.head(1)

,user_id,items_count,steam_id,user_url,item_id,item_name,playtime_forever,playtime_2weeks,0
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,10.0,Counter-Strike,6.0,0.0,NaN


In [6]:
df_items = df_items.drop(['0'], axis=1)

In [7]:
df_items.shape

(4931615, 8)

In [10]:
df_items['playtime_forever'].isna().sum()

12910

Intenté reemplazar los nulos de 'playtime_forever' con los valores de 'playtime_2weeks' si los tenía, pero no reemplazó ninguno, así que voy a borrar directamente los nulos de 'playtime_forever'.

In [9]:
df_items['playtime_forever'].fillna(df_items['playtime_2weeks'], inplace=True)

In [11]:
df_items.dropna(subset=['playtime_forever'], inplace=True)

Como no tengo nulos en 'user_id' voy a borrar 'steam_id' y 'user_url'. También voy a cambiar el tipo de dato a int.

In [12]:
steam_id = df_items['steam_id']
user_url = df_items['user_url']

In [13]:
df_items.drop(columns=['steam_id', 'user_url'], axis=1, inplace=True)

In [14]:
df_items['item_id'] = df_items['item_id'].astype(int)

In [15]:
type(df_items['item_id'][0])

numpy.int32

Transformación del texto a todo minúsculas y limpieza de caracteres no alfanumericos.

In [16]:
df_items = df_items.apply(lambda x: x.astype(str).str.lower() if x.dtype == "object" else x)

df_items = df_items.apply(lambda x: x.str.replace('-', ' ') if x.dtype == "object" else x)

df_items = df_items.apply(lambda x: x.str.replace('!', '') if x.dtype == "object" else x)

df_items = df_items.apply(lambda x: x.str.replace('¡', '') if x.dtype == "object" else x)

df_items = df_items.apply(lambda x: x.str.replace('?', '') if x.dtype == "object" else x)

df_items = df_items.apply(lambda x: x.str.replace('¿', '') if x.dtype == "object" else x)

df_items = df_items.apply(lambda x: x.str.replace('"', '') if x.dtype == "object" else x)

df_items = df_items.apply(lambda x: x.str.replace("'", '') if x.dtype == "object" else x)

df_items = df_items.apply(lambda x: x.str.replace(",", '') if x.dtype == "object" else x)

df_items = df_items.apply(lambda x: x.str.replace("&", '') if x.dtype == "object" else x)

df_items = df_items.apply(lambda x: x.str.replace("''", '') if x.dtype == "object" else x)

df_items = df_items.apply(lambda x: x.str.replace("_", '') if x.dtype == "object" else x)

In [17]:
df_items.head(3)

,user_id,items_count,item_id,item_name,playtime_forever,playtime_2weeks
0,76561197970982479,277,10,counter strike,6.0,0.0
1,76561197970982479,277,20,team fortress classic,0.0,0.0
2,76561197970982479,277,30,day of defeat,7.0,0.0


La columna 'playtime_2weeks' ya no me sirve. La voy a borrar.

In [18]:
playtime_2weeks = df_items['playtime_2weeks']

In [19]:
df_items.drop(columns='playtime_2weeks', axis=1, inplace=True)

Hay muchos outliers de 'playtime_forever', algunos de los cuales son imposibles. Intenté reemplazarlos con la mediana pero 
no me funcionó. Voy a borrarlos.

Hay muchos outliers de 'playtime_forever', algunos de los cuales son imposibles. Los reemplacé con la mediana.

In [ ]:
# Detectamos outliers de la columna 'playtime_forever' 
q1 = df_items['playtime_forever'].quantile(0.25)
q3 = df_items['playtime_forever'].quantile(0.75)
iqr = q3 - q1

In [ ]:
df_items.loc[df_items['playtime_forever'] > (q3 + 1.5 * iqr), 'playtime_forever'] = df_items['playtime_forever'].median()

In [21]:
df_items.value_counts()

user_id            items_count  item_id  item_name                     playtime_forever
x03 suits          141          1900     earth 2160                    1.0                 3
76561198081666970  46           407530   ark: survival of the fittest  0.0                 3
76561198027488037  53           10       counter strike                0.0                 3
76561198094973305  26           324810   toxikk                        0.0                 3
                                304930   unturned                      5.0                 3
                                                                                          ..
76561198073201110  65           319630   life is strange™              118.0               1
                                322330   dont starve together          134.0               1
                                323370   tera                          520.0               1
                                333930   dirty bomb                    13.0

In [24]:
import pyarrow.parquet as pq
df_items.to_parquet('Datos/df_items_limpio_compr.parquet')